# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
#get API Key
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not(api_key and api_key.startswith('sk-proj-') and len(api_key)>10):
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    

In [3]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

openai = OpenAI()
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


# set up environment

System prompt and user prompt

In [4]:
system_prompt = "You are very clever helper helping a student for any of his queries. \
You are to be guiding and encouraging to the question, \
and always give example to allow student to better understand.\n"

In [5]:
def user_prompt_for(question):
    user_prompt = f"You are understanding the question {question}"
    user_prompt += "please provide detail guide in markdown. \
That explain the answer to the question and \
Provide detailed example to enforce the learning. \n\n"
    return user_prompt

In [6]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [7]:
# Get gpt-4o-mini to answer, with streaming
def stream_GPT(question):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt_for(question)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [8]:
stream_GPT(question)

Certainly! Let's break down the code you provided step by step and provide an example to ensure understanding. Here's a detailed guide in  format.

# Understanding the Code: `yield from {book.get("author") for book in books if book.get("author")}`

## Introduction
The code snippet uses Python's `yield from`, a feature that allows a generator to yield all values from an iterable. In this case, it is yielding from a set comprehension that extracts authors from a list of books. 

Before diving deep, let's clarify some components:

1. **Generators**: Functions that allow you to make an iterator in a more Pythonic way, using the `yield` keyword instead of `return`.
2. **Set Comprehension**: A concise way to create sets in Python. It’s similar to list comprehension but creates a set instead, which means all items are unique.
3. **`get` Method**: This is a dictionary method that retrieves a value associated with a specified key. If that key does not exist, it returns `None` (or a default value if specified).

## Code Breakdown

### Step 1: The Set Comprehension
python
{book.get("author") for book in books if book.get("author")}

- **`for book in books`**: Loops through each `book` in the list `books`.
- **`book.get("author")`**: Fetches the value associated with the key `"author"` from each `book` dictionary.
- **`if book.get("author")`**: Ensures that only books that have an author (i.e., the author is not `None` or empty) are included in the set.

### Step 2: Yielding from the Set
- **`yield from`**: This keyword is used to yield all values produced by an iterable. By yielding from the resulting set, the generator returns each unique author found in the books.

## Complete Example

Let's illustrate with a complete example:

python
def get_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Example list of books
books = [
    {"title": "Book One", "author": "Author A"},
    {"title": "Book Two", "author": "Author B"},
    {"title": "Book Three", "author": None},
    {"title": "Book Four", "author": "Author A"},
    {"title": "Book Five", "author": ""},
]

# Using the generator to get authors
for author in get_authors(books):
    print(author)


### Expected Output:

Author A
Author B


### Explanation of the Example:

1. **Books List**: A list of dictionaries where each dictionary represents a book with a title and an author.
2. **Function Definition**: We defined a generator function `get_authors(books)` that extracts authors from the `books` list.
3. **Yielding Unique Authors**: The set comprehension:
    - Collects authors, eliminating duplicates.
    - Ignores books with no valid author.
4. **Output**: When calling `get_authors(books)`, the loop prints the unique authors, `Author A` and `Author B`.

## Conclusion
This code is an efficient way to gather unique authors from a collection of books while ignoring those without an associated author. Using `yield from` with set comprehension helps keep the code clean and efficient. 

By practicing with different datasets, you'll get more comfortable with comprehensions and generators in Python! Feel free to ask if you need any more examples or further clarification!

In [9]:
# Get Llama 3.2 to answer

def stream_Ollama(question):
    stream = ollama_via_openai.chat.completions.create(
        model = MODEL_LLAMA,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt_for(question)}
          ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [10]:
stream_Ollama(question)

**Generator Expression Explanation**
======================================

### Overview

The given code snippet is a generator expression that uses `yield from` to iterate over specific data from an iterable object called `books`. We'll break down what each part does.

### Code Breakdown

python
yield from {book.get("author") for book in books if book.get("author")}


- **dict comprehension**: `<expr> for <iterable> if condition`
  - Creates a new dictionary with the expression `book.get("author")` applied to each element in the iterable `books`.
  - The `{}` denotes a new dictionary.

### Step-by-Step Explanation

1. `books`: This is an iterable (likely a list or other collection).
2. `for book in books if book.get("author")`: This part filters out elements from the `books` iterable where `"author"` does not exist as a key.
3. `{book.get("author") for book in books if book.get("author")}`: Within each remaining element, this expression uses `get()` to retrieve the `"author"` value inside that dictionary and yields it as an individual element in the new dictionary.

### Behavior

When you use `yield from`, instead of generating a single iterable or collection with those dictionary values, Python will 'generate' (i.e., yield one item at a time) each author name found within your books:

- **`yield from`**: A part that turns another iterable into an iterator. 
    - In this case, it tells Python to take every individual value and add them to the final output.

**Detailed Example**
-------------------

Let's assume we are working with two dictionaries (books) where each dictionary contains book information (`title` & `author`):

python
books = [
    {"id": 1, "title": "Book A", "author": "Author 1"},
    {"id": 2, "title": "Book B", "author": None},
    {"id": 3, "title": "Book C", "author": "Author 3"}
]

book_author_values = {book.get("author") for book in books if book.get("author")}


Here results the `{ 'Author 1', 'Author 3' }` because `None` was ignored from `" author" == None`.